In [1]:
import numpy as np
import math

## Method

In [2]:
# calculate optimal sample allocation for Monte Carlo (MC)
def MC(n_levels, V, epsilon_squared):
    N_GPU = np.zeros(n_levels)
    N_GPU[-1] = V[-1]/epsilon_squared
    return N_GPU

In [3]:
# calculate optimal sample allocation for Multilevel Monte Carlo (MLMC)
# M.B. Giles, 2018, Multilevel Monte Carlo methods, Acta Numerica, https://people.nps.ox.ac.uk/gilesm/files/acta15.pdf
def MLMC(n_levels, V, C_GPU, epsilon_squared):
    N_GPU = np.zeros(n_levels)
    mu = 0.
    for l in range(n_levels):
        mu += np.sqrt(V[l]*C_GPU[l])
    mu /= epsilon_squared
    N_GPU = mu * np.sqrt(V/C_GPU)
    return N_GPU

In [4]:
# calculate optimal sample allocation for Multilevel Monte Carlo on Heterogeneous Computer Architectures (hMLMC)
# C. Adcock, L. Jofre, G. Iaccarino, 2020, Multilevel Monte Carlo Sampling on Heterogenous Computer Architectures, 
#   International Journal for Uncertainty Quantification, 
#   http://www.dl.begellhouse.com.stanford.idm.oclc.org/journals/52034eb04b657aea,3673619972b2eee6,2dc1fd2d0ec3d1c0.html
def hMLMC(n_levels, n_CPUs, n_GPUs, C_CPU, V, C_GPU, epsilon_squared):
    # Step 1
    alphas = np.sort(C_CPU/C_GPU*n_GPUs/n_CPUs, kind = 'quicksort') #quicksort is default
    alpha_lb = alphas[0]
    alpha_ub = alphas[n_levels - 1]
    finished = False

    while not finished:
        # Step 2
        alpha = (alpha_lb + alpha_ub)/2.
        
        # Step 3         
        lambda_2 = 1./(1. + alpha)
        lambda_3 = alpha/(1. + alpha)

        # Step 4
        lambda_1 = 0.
        for l in range(n_levels):
            lambda_1 += np.sqrt(V[l]/np.maximum(1./(lambda_2*C_CPU[l]*n_GPUs), \
                                            1./(lambda_3*C_GPU[l]*n_CPUs)))
        lambda_1 /= epsilon_squared
        lambda_1 = lambda_1**2

        # Step 5
        N_CPU = np.zeros(n_levels)
        N_GPU = np.zeros(n_levels)

        L_free = []
        for l in range(n_levels):     
            if math.isclose(lambda_3/lambda_2, C_CPU[l]/C_GPU[l]*n_GPUs/n_CPUs, rel_tol=1e-6):
                L_free.append(l)          
            elif lambda_3/lambda_2 > C_CPU[l]/C_GPU[l]*n_GPUs/n_CPUs:
                # still N_GPU[l] = 0.
                N_CPU[l] = np.sqrt((lambda_1*V[l])/(lambda_2*C_CPU[l]*n_GPUs))
            else:
                # lambda_3/lambda_2 < C_CPU[l]/C_GPU[l]*n_GPUs/n_CPUs:
                N_GPU[l] = np.sqrt((lambda_1*V[l])/(lambda_3*C_GPU[l]*n_CPUs))
                # still N_CPU[l] = 0.

        # balance CPU and GPU cost using underconstrained N_l^C, N_l^G variables
        # while ensuring N_l^C>0, N_l^G>0, and total variance = epsilon^2
        for l in L_free:
            C_CPU_total = np.sum(N_CPU*C_CPU)/n_CPUs
            C_GPU_total = np.sum(N_GPU*C_GPU)/n_GPUs
            N_l = np.sqrt((lambda_1*V[l])/(lambda_2*C_CPU[l]*n_GPUs))
            # equivalently N_l = np.sqrt((lambda_1*V[l])/(lambda_3*C_GPU[l]))

            if math.isclose(C_CPU_total, C_GPU_total):
                N_CPU[l] = (N_l*C_GPU[l]*n_CPUs)/(C_CPU[l]*n_GPUs+C_GPU[l]*n_CPUs)
                N_GPU[l] = (N_l*C_CPU[l]*n_GPUs)/(C_CPU[l]*n_GPUs+C_GPU[l]*n_CPUs)
            elif C_CPU_total > C_GPU_total:
                N_GPU_excess = n_GPUs/C_GPU[l]*(C_CPU_total - C_GPU_total)              
                if N_GPU_excess >= N_l:
                    N_GPU[l] = N_l
                    # still N_CPU[l] = 0.
                else:
                    N_GPU[l] = N_GPU_excess
                    N_to_split = N_l - N_GPU[l]
                    N_CPU[l] = (N_to_split*C_GPU[l]*n_CPUs)/(C_CPU[l]*n_GPUs+C_GPU[l]*n_CPUs)
                    N_GPU[l] += (N_to_split*C_CPU[l]*n_GPUs)/(C_CPU[l]*n_GPUs+C_GPU[l]*n_CPUs)
            else:
                # C_CPU_total < C_GPU_total
                N_CPU_excess = n_CPUs/C_CPU[l]*(C_GPU_total - C_CPU_total)
                if N_CPU_excess >= N_l:
                    N_CPU[l] = N_l
                    # still N_CPU[l] = 0.
                else:
                    N_CPU[l] = N_CPU_excess
                    N_to_split = N_l - N_CPU[l]
                    N_CPU[l] += (N_to_split*C_GPU[l]*n_CPUs)/(C_CPU[l]*n_GPUs+C_GPU[l]*n_CPUs)
                    N_GPU[l] = (N_to_split*C_CPU[l]*n_GPUs)/(C_CPU[l]*n_GPUs+C_GPU[l]*n_CPUs)
                    
        # Step 6
        C_CPU_total = np.sum(N_CPU*C_CPU)/n_CPUs
        C_GPU_total = np.sum(N_GPU*C_GPU)/n_GPUs
        
        # Exit or prep for next iteration
        if math.isclose(C_CPU_total, C_GPU_total):        
            finished = True
            break
        elif C_CPU_total > C_GPU_total:
            alpha_ub = alpha
        else:
            # C_CPU_total < C_GPU_total
            alpha_lb = alpha
    
    return N_CPU, N_GPU

## Test Case
Channel flow with stochastic heat flux BC (SoleilX)

C. Adcock, L. Jofre, G. Iaccarino, 2020, Multilevel Monte Carlo Sampling on Heterogenous Computer Architectures, International Journal for Uncertainty Quantification, http://www.dl.begellhouse.com.stanford.idm.oclc.org/journals/52034eb04b657aea,3673619972b2eee6,2dc1fd2d0ec3d1c0.html

In [5]:
n_CPUs = 792.*44.
n_GPUs = 792.*4.

# levels created by coarsening grid
# grid points for each level: 64x32, 128x64, 128x128, 256x128, 512x128
n_levels_all = 5

# cost of running a sample on a CPU for each level
C_CPU_all = np.array([628.602, 4389.38, 15448.9, 74334.2, 3.400593e+05])

# cost of running a sample on a GPU for each level
C_GPU_all = np.array([636.529, 2273.46, 4602.59, 11595.9, 2.804512e+04])

# variance of each level, V[Q_l], where Q_l is a sample on level l
V_Q_all = np.array([0.000182002, 0.000143691, 0.000108583, 0.000160483, 1.316855e-04])

# variance of the difference between each level and the preceding level, V[Q_l - Q_{l-1}]
V_dQ_all = np.array([np.nan, 1.64063e-05, 1.30941e-06, 1.05553e-06, 3.193399e-07])

# tolerance on the variance
epsilon_squared = 1e-6

In [6]:
print('C_CPU/C_GPU: ', np.round(C_CPU_all/C_GPU_all, 2))
print('')
for i in range(n_levels_all):
    C_CPU = C_CPU_all[i:].copy()
    C_GPU = C_GPU_all[i:].copy()
    V = V_dQ_all[i:].copy()
    V[0] = V_Q_all[i].copy()
    
    n_levels = n_levels_all-i
    print('Using finest', n_levels, 'levels:')
    
    N_CPU_hMLMC, N_GPU_hMLMC = hMLMC(n_levels, n_CPUs, n_GPUs, C_CPU, V, C_GPU, epsilon_squared)
 
    # number of samples to run on GPUs on each level for MLMC sample allocation
    # note 1: this isn't necessarily just N_CPU_hMLMC + N_GPU_hMLMC
    # note 2: MLMC assumes only one processor type; here assuming GPUs used since they're
    #   typically faster than GPUs
    N_GPU_MLMC = MLMC(n_levels, V, C_GPU, epsilon_squared)
    print('N_GPU_MLMC: ', np.round(N_GPU_MLMC))
    
    # number of samples to run on CPUs on each level for hMLMC sample allocation
    #   leftmost position is coarsest level
    print('N_CPU_hMLMC: ', np.round(N_CPU_hMLMC))
    
    # number of samples to run on GPUs on each level for hMLMC sample allocation
    print('N_GPU_hMLMC: ', np.round(N_GPU_hMLMC))

    # variance--check that <= tolerance (epsilon_squared)
    print('tolerance: ', epsilon_squared)
    print ('V_MLMC: ', np.round(sum(V/(np.round(N_GPU_MLMC))), 8))
    print('V_hMLMC: ', np.round(sum(V/(np.round(N_CPU_hMLMC)+np.round(N_GPU_hMLMC))), 8))
    
    # wall-clock time-to-solution
    C_hMLMC = np.max([np.sum(C_CPU*N_CPU_hMLMC)/n_CPUs, np.sum(C_GPU*N_GPU_hMLMC)/n_GPUs])
    C_MLMC = np.sum(C_GPU*N_GPU_MLMC)/n_GPUs
    print('T_MLMC: ', np.round(C_MLMC, 0))
    print('T_hMLMC: ', np.round(C_hMLMC, 0))
    print('T_hMLMC/T_MLMC: ', np.round(C_hMLMC/C_MLMC,2))
    print('')

C_CPU/C_GPU:  [ 0.99  1.93  3.36  6.41 12.13]

Using finest 5 levels:
N_GPU_MLMC:  [437.  69.  14.   8.   3.]
N_CPU_hMLMC:  [683.  78.  12.   1.   0.]
N_GPU_hMLMC:  [0. 0. 0. 4. 2.]
tolerance:  1e-06
V_MLMC:  9.9e-07
V_hMLMC:  9.6e-07
T_MLMC:  210.0
T_hMLMC:  29.0
T_hMLMC/T_MLMC:  0.14

Using finest 4 levels:
N_GPU_MLMC:  [215.  14.   8.   3.]
N_CPU_hMLMC:  [259.  13.   0.   0.]
N_GPU_hMLMC:  [0. 0. 6. 2.]
tolerance:  1e-06
V_MLMC:  1e-06
V_hMLMC:  9.9e-07
T_MLMC:  230.0
T_hMLMC:  38.0
T_hMLMC/T_MLMC:  0.17

Using finest 3 levels:
N_GPU_MLMC:  [140.   9.   3.]
N_CPU_hMLMC:  [139.   0.   0.]
N_GPU_hMLMC:  [2. 9. 3.]
tolerance:  1e-06
V_MLMC:  1e-06
V_hMLMC:  9.9e-07
T_MLMC:  263.0
T_hMLMC:  61.0
T_hMLMC/T_MLMC:  0.23

Using finest 2 levels:
N_GPU_MLMC:  [172.   5.]
N_CPU_hMLMC:  [116.   0.]
N_GPU_hMLMC:  [56.  5.]
tolerance:  1e-06
V_MLMC:  1e-06
V_hMLMC:  1e-06
T_MLMC:  672.0
T_hMLMC:  247.0
T_hMLMC/T_MLMC:  0.37

Using finest 1 levels:
N_GPU_MLMC:  [132.]
N_CPU_hMLMC:  [63.]
N_GPU_hML